In [1]:
!pip install openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 772.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.7 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

ds = load_dataset("argilla/llama-2-banking-fine-tune",split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
len(ds)

100

In [5]:
ds[0]

{'request': 'I tried to make a transfer but it failed',
 'response-1': " Thank you for choosing the Bank of America for your financial needs! I'm so sorry to hear that your transfer failed. Can you please provide me with more details about the transfer, such as the amount and the account it was being transferred from/to? This will help me better understand the issue and assist you in resolving it as quickly as possible. Additionally, have you tried checking your account balance or recent transactions to see if the transfer has been processed? If you have any further questions or concerns, please don't hesitate to ask. Thank you for trusting the Bank of America with your financial matters!",
 'response-2': " Hello there! Thank you for choosing the Bank of America for your financial needs. I'm so sorry to hear that your transfer failed. Can you please provide me with more details about the transfer, such as the amount and the account it was being transferred from/to? Additionally, do you

In [7]:
ds_formatted = [
    {"messages": [
        {"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language."},
        {'role': 'user', 'content': x["request"]},
        {'role': 'assistant', 'content': x["response-1"]}]} for x in ds
]

In [8]:
ds_formatted[0]

{'messages': [{'role': 'system',
   'content': 'You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language.'},
  {'role': 'user', 'content': 'I tried to make a transfer but it failed'},
  {'role': 'assistant',
   'content': " Thank you for choosing the Bank of America for your financial needs! I'm so sorry to hear that your transfer failed. Can you please provide me with more details about the transfer, such as the amount and the account it was being transferred from/to? This will help me better understand the issue and assist you in resolving it as quickly as possible. Additionally, have you tried checking your account balance or recent transactions to see if the transfer has been processed? If you have any further questions or concerns, please don't hesitate to ask. Thank you for trusting the Bank of America with your financial matters!"}]}

In [9]:
import random
random.shuffle(ds_formatted)

In [10]:
ds_train = ds_formatted[:80]
ds_val = ds_formatted[80:]

In [13]:
import json

with open('train.jsonl', 'w') as f:
    for line in ds_train:
        json.dump(line, f)
        f.write('\n')

with open('val.jsonl', 'w') as f:
    for line in ds_val:
        json.dump(line, f)
        f.write('\n')

In [14]:
ls

sample_data/  train.jsonl  val.jsonl


In [15]:
!head -n 2 train.jsonl

{"messages": [{"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language."}, {"role": "user", "content": "Where can I get my card at?"}, {"role": "assistant", "content": " Hello there! Thank you for choosing the Bank of America for your banking needs. I'm happy to help you with your inquiry. If you're looking to obtain a new card, you can visit any of our banking centers or ATMs located throughout the country. You can also request a card through our online banking platform or by giving us a call at our 24/7 customer service line.\nTo obtain a new card, please bring a valid form of identification and proof of address to the banking center or ATM. Our staff will be happy to assist you with the process. If you have any further questions or concerns, please don't hesitate to ask. Thank you for trusting the Bank of America with your banking needs."}]}
{"messages": [{"role": "system", "con

In [16]:
!head -n 2 val.jsonl

{"messages": [{"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language."}, {"role": "user", "content": "I think something went wrong with my card delivery as I haven't received it yet."}, {"role": "assistant", "content": " Hello there! Thank you for reaching out to us at the Bank of America. We apologize for any inconvenience you're experiencing with your card delivery. Can you please provide me with your full name and the last four digits of your account number so I can look into this for you? Additionally, can you tell me the date you placed the order and the expected delivery date? This information will help me locate your order and assist you in resolving the issue. Thank you for trusting the Bank of America with your banking needs."}]}
{"messages": [{"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests usi

# Upload data

In [2]:
import openai

openai.api_key = 'TYPE YOUR OWN API KEY'


In [17]:
# Upload training data
train = openai.File.create(
  file=open("train.jsonl", "rb"),
  purpose='fine-tune'
)
train

<File file id=file-7a9HvTwKmYmwWXLjW4wz07mw at 0x7eedf2d78ef0> JSON: {
  "object": "file",
  "id": "file-7a9HvTwKmYmwWXLjW4wz07mw",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 88257,
  "created_at": 1692821696,
  "status": "uploaded",
  "status_details": null
}

In [18]:
train_id = train['id']

In [19]:
# Upload validation data
val = openai.File.create(
  file=open("val.jsonl", "rb"),
  purpose='fine-tune'
)
val

<File file id=file-AdZ5IWjD1QfpNOuKJ9Br84Oh at 0x7eee009fe980> JSON: {
  "object": "file",
  "id": "file-AdZ5IWjD1QfpNOuKJ9Br84Oh",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 20669,
  "created_at": 1692821718,
  "status": "uploaded",
  "status_details": null
}

In [20]:
val_id = val['id']

# Fine-tuning

In [24]:
# Create a fine-tuned model
response = openai.FineTuningJob.create(
    training_file=train_id,
    validation_file=val_id,
    model="gpt-3.5-turbo"
    )
response

<FineTuningJob fine_tuning.job id=ftjob-EyEBJvq3gIvLVc9zc49kB1dv at 0x7eee00a2cf90> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-EyEBJvq3gIvLVc9zc49kB1dv",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692821828,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-YOkCoqnAhDkMBvC9xTc09wYD",
  "result_files": [],
  "status": "created",
  "validation_file": "file-AdZ5IWjD1QfpNOuKJ9Br84Oh",
  "training_file": "file-7a9HvTwKmYmwWXLjW4wz07mw",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [25]:
job_id = response['id']

In [26]:
# Retrieve the state of a fine-tune
response = openai.FineTuningJob.retrieve(job_id)
response

<FineTuningJob fine_tuning.job id=ftjob-EyEBJvq3gIvLVc9zc49kB1dv at 0x7eee2c9bb100> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-EyEBJvq3gIvLVc9zc49kB1dv",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692821828,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-YOkCoqnAhDkMBvC9xTc09wYD",
  "result_files": [],
  "status": "running",
  "validation_file": "file-AdZ5IWjD1QfpNOuKJ9Br84Oh",
  "training_file": "file-7a9HvTwKmYmwWXLjW4wz07mw",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [27]:
# Retrieve the state of a fine-tune
response = openai.FineTuningJob.retrieve(job_id)
response

<FineTuningJob fine_tuning.job id=ftjob-EyEBJvq3gIvLVc9zc49kB1dv at 0x7eedf2c225c0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-EyEBJvq3gIvLVc9zc49kB1dv",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692821828,
  "finished_at": 1692822626,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7qomeQF9",
  "organization_id": "org-YOkCoqnAhDkMBvC9xTc09wYD",
  "result_files": [
    "file-NW17EY2UjdeoUMhGdpuFDt55"
  ],
  "status": "succeeded",
  "validation_file": "file-AdZ5IWjD1QfpNOuKJ9Br84Oh",
  "training_file": "file-7a9HvTwKmYmwWXLjW4wz07mw",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 49788
}

In [28]:
# List up to 10 events from a fine-tuning job
response = openai.FineTuningJob.list_events(id=job_id, limit=10)
response

<OpenAIObject list at 0x7eedf2da0180> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-augZ8gjXmiMHpNL62LOnMAoD",
      "created_at": 1692822626,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-0Y27ZDZ6P1tw83r70O9AuAHk",
      "created_at": 1692822624,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::7qomeQF9",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-ryQ8taNMxlj71KDwVIhHyq1D",
      "created_at": 1692822620,
      "level": "info",
      "message": "Step 240: training loss=0.33",
      "data": {
        "step": 240,
        "train_loss": 0.32981249690055847,
        "train_mean_token_accuracy": 0.8760330677032471
      },
      "type": "metrics"
  

In [29]:
events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Step 170: training loss=0.42
Step 180: training loss=0.64
Step 190: training loss=0.31
Step 200: training loss=0.41
Step 210: training loss=0.41
Step 220: training loss=0.45
Step 230: training loss=0.64
Step 240: training loss=0.33
New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::7qomeQF9
Fine-tuning job successfully completed


In [30]:
response = openai.FineTuningJob.retrieve(job_id)
response

<FineTuningJob fine_tuning.job id=ftjob-EyEBJvq3gIvLVc9zc49kB1dv at 0x7eedf2c0c6d0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-EyEBJvq3gIvLVc9zc49kB1dv",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692821828,
  "finished_at": 1692822626,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:personal::7qomeQF9",
  "organization_id": "org-YOkCoqnAhDkMBvC9xTc09wYD",
  "result_files": [
    "file-NW17EY2UjdeoUMhGdpuFDt55"
  ],
  "status": "succeeded",
  "validation_file": "file-AdZ5IWjD1QfpNOuKJ9Br84Oh",
  "training_file": "file-7a9HvTwKmYmwWXLjW4wz07mw",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 49788
}

In [31]:
model_id = response["fine_tuned_model"]

# Inference

In [32]:
test_messages = [
    {"role": "system", "content": "You are a customer service representative from Bank of America. Please reply customer requests using polite and respectful language."},
    {"role": "user", "content": "My card is stolen!"}
  ]

In [37]:
completion = openai.ChatCompletion.create(
  model=model_id,
  messages=test_messages
)


In [38]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": " I'm so sorry to hear that your card has been stolen! Thank you for trusting the Bank of America with your financial well-being. I'm here to help you resolve this issue as quickly as possible. Could you please provide me with your name and account number so that I can verify your information and help you process a claim? Additionally, please let me know when and where the theft occurred so that we can take appropriate action. Thank you for your cooperation and apologies for any inconvenience this may have caused you."
}


In [39]:
completion1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=test_messages
)


In [40]:
print(completion1.choices[0].message)

{
  "role": "assistant",
  "content": "I'm sorry to hear that your card has been stolen. I understand the urgency of the situation. Please remain calm and take the following steps:\n\n1. Contact our customer service immediately at 1-800-432-1000 to report the theft. Our representatives will assist you in blocking your card to prevent any unauthorized transactions.\n\n2. Monitor your account activity closely to detect any fraudulent charges. If you notice any suspicious transactions, please inform us promptly so that we can initiate the dispute process and protect your funds.\n\n3. Once you've made an official report, we will guide you through the process of ordering a replacement card. It will typically be delivered to your mailing address within 7-10 business days.\n\nI understand that this situation can be stressful, but be assured that our team is here to support you and help resolve any issues caused by the theft."
}
